In [1]:
#!/usr/bin/env python3
import os
import logging
import optparse
from datetime import datetime
import requests
import json

In [2]:
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

In [ ]:
def process(options):

    logging.info("Testing ES ... ")
    url = options.server + "/" + options.index
    print(url)
    headers = {'content-type': 'application/json'}
    res=requests.get(url)
    logging.info("Reading input file ... ")    
    data = json.load(open(options.input, "r"))
    assert len(data) > 0
    logging.info("Creating index ... ")
    # first if type is available, we need to map it to a keyword
    row = data[0]
    if 'type' in row.keys() and options.mapping != "yes":
        create_index = """
            {
                "mappings": {
                    "_doc": {
                    "properties": {
                      "type": { "type": "keyword" }
                    }
                    }
                }
            }
            """
        res = requests.put(url, create_index, headers=headers).text
        logging.info(res)
    elif options.mapping == "yes":
        create_index = """
            {
                "mappings": {
                    "_doc": {
                    "dynamic_templates": [
                            {
                                "strings_as_keywords": {
                                  "match_mapping_type": "string",
                                  "mapping": {
                                   "type": "keyword"
                                  }
                            }
                          }

                        ],                    
                    "properties": {
                       "type": { "type": "keyword" },
                       "text": { "type": "text" }
                    }
                    }
                }
            }
            """
        res = requests.put(url, create_index, headers=headers).text
        logging.info(res)
    logging.info("Loading data ... ")
    for i in range(len(data)):
        doc = data[i]
        idx = str(i)
        if 'id' in doc.keys():
            idx = str(doc['id'])
        try:
            res = requests.post(url=url+"/_doc/"+idx,json=doc)
            if res.status_code < 200 or res.status_code > 299:
                logging.error(res.text)
        except Exception as inst:
            logging.error("Cannot process document %d" %i)
            logging.error(type(inst))
            logging.error(inst)
            logging.error(d)
            
        if i % 1000 == 0:
            logging.info("%d documents processed" % i)   
    logging.info("%d documents processed" % i)
    logging.info("Done!")

In [3]:
def main():
    usage = "usage: %prog [options]"
    parser = optparse.OptionParser(usage)
    parser.add_option("-i", "--input",  dest="input",
                    action="store", type="string", default="out.json",
                    help="""input file name, .json file generated by preprocessor
                            default is out.json""")

    parser.add_option("-m", "--mapping", dest="mapping",
                     action="store", type="string", default="yes",
                     help="create index with default mappings, default is yes")

    parser.add_option("-s", "--server", dest="server",
                     action="store", type="string", default="http://localhost:9200",
                     help="elasticsearch url, default is http://localhost:9200")
    
    parser.add_option("-x", "--index", dest="index",
                     action="store", type="string", default="documents",
                     help="elasticsearch index name, default is documents")
     
    options, args = parser.parse_args()
    
    process(options)

In [ ]:
if __name__ == "__main__":
    main()  
    exit()

In [5]:
!jupyter nbconvert --to script indexer.ipynb

[NbConvertApp] Converting notebook indexer.ipynb to script
[NbConvertApp] Writing 3991 bytes to indexer.py
